In [ ]:
import ipywidgets as widgets
from IPython.display import display
import requests

class CreateUser:
    def __init__(self, host, output):
        self.host = host
        self.out = output
        self.first_name_textbox =widgets.Text(
            placeholder='Enter your First Name',
            description='First Name:',
            disabled=False)
        
        self.last_name_textbox =widgets.Text(
            placeholder='Enter your Last Name',
            description='Last Name:',
            disabled=False) 

        self.age_boundedinttext = widgets.BoundedIntText(
            min=1,
            max=200,
            step=1,
            description='Age:',
            disabled=False
        )

        self.position_dropdown = widgets.Dropdown(
            options=['Regular', 'Supervisor', 'Administrator'],
            description='Position:',
            disabled=False,
        )

        self.salary_boundedfloattext = widgets.BoundedFloatText(
            min=0,
            max=10000000,
            step=0.01,
            description='Salary:',
            disabled=False
        )

        self.username_textbox = widgets.Text(
            placeholder='Enter your Username',
            description='Username:',
            disabled=False   
        )

        hotels = requests.get(f"{self.host}/deceptecons/hotel").json()
        hotels.sort(key=lambda x: x['hid'])
        hotels = list(map(lambda x: [f"{x['hid']}: {x['hname']}", x['hid']], hotels))

        self.hotels_dropdown = widgets.Dropdown(
            options=hotels,
            description='Hotel:',
            disabled=False
        )

        self.password_textbox = widgets.Password(
            placeholder='Enter a password',
            description='Password:',
            disabled=False
        ) 

        self.submit_account_button = widgets.Button(
            description='Submit:',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Submit',
            icon='check' # (FontAwesome names without the `fa-` prefix)
        )

        self.submit_account_button.on_click(self.submit_account)

    def submit_account(self, b):
        if self.first_name_textbox.value != "" and \
        self.last_name_textbox.value != "" and \
        self.age_boundedinttext.value is not None and\
        self.position_dropdown.value is not None and \
        self.salary_boundedfloattext.value is not None and \
        self.username_textbox.value != "" and \
        self.password_textbox.value != "" and\
        self.hotels_dropdown.value is not None:
            fname = self.first_name_textbox.value
            lname = self.last_name_textbox.value
            age = self.age_boundedinttext.value
            position = self.position_dropdown.value
            salary = self.salary_boundedfloattext.value
            hid = self.hotels_dropdown.value
            if position == 'Administrator':
                hid = -1
            elif hid == -1:
                position = 'Administrator'
            data = {'hid': hid, 'fname': fname, 'lname': lname, 'age': age, 'position': position, 'salary': salary}
            eid = int(requests.post(f"{self.host}/deceptecons/employee", json=data).json())
            username = self.username_textbox.value
            password = self.password_textbox.value
            data = {'eid': eid, 'username': username, 'password': password}
            lid = int(requests.post(f"{self.host}/deceptecons/login", json=data).json())
            user = dict()
            user['eid'] = eid
            user['lid'] = lid
            user['pass'] = password
            user['user'] = username
            user['hid'] = hid
            user['position'] = position
            self.hideCreateUser()
            %run AggregateSelectorDasboard.ipynb
            run(user, self.host, self.out)

    def displayCreateUser(self):
        with self.out:
            display(widgets.VBox([
                self.first_name_textbox,
                self.last_name_textbox,
                self.age_boundedinttext,
                self.position_dropdown,
                self.salary_boundedfloattext,
                self.hotels_dropdown,
                self.username_textbox,
                self.password_textbox,
                self.submit_account_button
            ]))
    def hideCreateUser(self):
        self.first_name_textbox.close()
        self.last_name_textbox.close()
        self.age_boundedinttext.close()
        self.position_dropdown.close()
        self.salary_boundedfloattext.close()
        self.hotels_dropdown.close()
        self.username_textbox.close()
        self.password_textbox.close()
        self.submit_account_button.close()

def run(host, output):
    x = CreateUser(host, output)
    x.displayCreateUser()